* Importando os pecotess necessários

In [91]:
import urllib, urllib.error
from functions import save_in_database
from urllib import parse
import json
import requests
import sqlite3
import pandas as pd

* Define uma função que recebe um dicionário JSON e imprime informações específicas desse dicionário, desde que essas informações correspondam às chaves especificadas na lista list_keys. Ele cria uma apresentação organizada das informações do filme, separadas por hifens, para facilitar a leitura e compreensão.

In [92]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings',
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for i in list_keys:
        if i in list(json_data.keys()):
            print(f"{i}: {json_data[i]}")
    print("-"*50)

In [93]:
def search_movie(title):
    if len(title) < 1 or title=='quit': 
        print("Não encontrado...")
        return 

    try:
        title_url = parse.urlencode({'t': title})
        api_key = '9019e34'
        url = f'http://www.omdbapi.com/?{title_url}&apikey={api_key}'
        print(f'Procurando o filme "{title}"... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_database =input('Salvar o filme no banco de dados? Digite "sim" ou "não": ').lower()
            if save_database =='sim':
                save_in_database(json_data)
        else:
            print("Erro encontrado: ",json_data['Erro'])
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")

In [94]:
def save_in_database(movie_data):
    conn = sqlite3.connect('movies.db')
    cur=conn.cursor()

    cur.execute ('''CREATE TABLE IF NOT EXISTS filmes (
        title TEXT,
        year INTEGER,
        rated TEXT,
        released TEXT,
        runtime TEXT,
        genre TEXT,
        director TEXT,
        writer TEXT,
        actors TEXT,
        plot TEXT,
        language TEXT,
        country TEXT,
        awards TEXT,
        ratings TEXT,
        metascore INTEGER,
        imdbRating REAL,
        imdbVotes TEXT,
        imdbID TEXT
    )
''')

    conn.execute('INSERT INTO filmes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
             (movie_data['Title'], movie_data['Year'], movie_data['Rated'], movie_data['Released'], movie_data['Runtime'],
              movie_data['Genre'], movie_data['Director'], movie_data['Writer'], movie_data['Actors'], movie_data['Plot'],
              movie_data['Language'], movie_data['Country'], movie_data['Awards'], movie_data['Ratings'], movie_data['Metascore'],
              movie_data['imdbRating'], movie_data['imdbVotes'], movie_data['imdbID']))
    conn.commit()
    conn.close()

* Pegar a chave da API

In [95]:
title = input('\nDigite o nome do filme: ')
if len(title) < 1 or title=='quit': 
    print("Não encontrado...")
else:
    movie_data = search_movie(title)
    if movie_data is not None:
        save_in_database(movie_data)

Procurando o filme "Barbie"... 
--------------------------------------------------
Title: Barbie
Year: 2023
Rated: PG-13
Released: 21 Jul 2023
Runtime: 114 min
Genre: Adventure, Comedy, Fantasy
Director: Greta Gerwig
Writer: Greta Gerwig, Noah Baumbach
Actors: Margot Robbie, Ryan Gosling, Issa Rae
Plot: Barbie suffers a crisis that leads her to question her world and her existence.
Language: English, Spanish
Country: United States, United Kingdom
Awards: 2 wins & 1 nomination
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '88%'}, {'Source': 'Metacritic', 'Value': '80/100'}]
Metascore: 80
imdbRating: 7.4
imdbVotes: 238,040
imdbID: tt1517268
--------------------------------------------------


OperationalError: table filmes has 18 columns but 17 values were supplied